# Demonstration of the Entmoot API

This notebook includes comparisons of the Entmoot strategy to other Bofire 
strategies.

## Single objective Bayesian Optimization strategy

This section includes a comparison to the Sobo strategy given in the "Getting 
Started" docs. The API is identical, with additional options given to the 
data model.

In [ ]:
from bofire.benchmarks.single import Himmelblau
import bofire.strategies.mapper as strategy_mapper

benchmark = Himmelblau()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [ ]:
from bofire.data_models.strategies.api import EntingStrategy

enting_strategy_data_model = EntingStrategy(
    domain=benchmark.domain,
    dist_metric="l1",
    acq_sense="exploration",
    solver_name="gurobi",
    solver_verbose=False,
    kappa_fantasy=1.96
)

enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

When `candidate_count > 1`, the strategy will fit to each candidate in order to produce the next candidate. This means that sequential calls to `EntingStrategy.ask()` will tend to be further from the ground truth.

If you are using subsequent calls to `EntingStrategy.ask()`, then you must add the candidates to the pending list of candidates. Note that this is not advised - it is much faster to generate candidates in a single batch.

In [ ]:
a = enting_strategy.ask(candidate_count=5)
b = enting_strategy.ask(candidate_count=5, add_pending=True)
c = enting_strategy.ask(candidate_count=5)

assert a.equals(b)
assert not b.equals(c)

## Multi Objective BO

In [ ]:
from bofire.benchmarks.multi import DTLZ2
import bofire.strategies.mapper as strategy_mapper

benchmark = DTLZ2(dim=2, num_objectives=2)

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [ ]:
enting_strategy_data_model = EntingStrategy(
    domain=benchmark.domain,
    dist_metric="l1",
    acq_sense="exploration",
    solver_name="gurobi",
    solver_verbose=False,
    kappa_fantasy=1.96
)

enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

## Maximize single objective

In [ ]:
from bofire.benchmarks.single import Ackley

benchmark = Ackley()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [ ]:
enting_strategy_data_model = EntingStrategy(
    domain=benchmark.domain,
    dist_metric="l1",
    acq_sense="exploration",
    solver_name="gurobi",
    solver_verbose=False,
    kappa_fantasy=1.96
)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

## Compare domain to problem config

BoFire defines its problems using Domains, whereas ENTMOOT uses ProblemConfigs. 
This demonstrates the conversion from Domain to ProblemConfig, using the multi 
objective categorical problem defined in the ENTMOOT benchmark.

In [ ]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import (
    CategoricalInput,
    ContinuousInput,
    DiscreteInput,
    ContinuousOutput,
)
from bofire.data_models.objectives.api import MinimizeObjective


def build_multi_obj_categorical_problem(n_obj: int = 2, no_cat=False) -> Domain:
    """
    Builds a small test example which is used in Entmoot tests.
    """

    cat_feat = (
        []
        if no_cat
        else [CategoricalInput(key="x0", categories=("blue", "orange", "gray"))]
    )
    input_features = Inputs(
        features=cat_feat
        + [
            DiscreteInput(key="x1", values=[5, 6]),
            DiscreteInput(key="x2", values=[0, 1]),  # binary
            ContinuousInput(key="x3", bounds=[5.0, 6.0]),
            ContinuousInput(key="x4", bounds=[4.6, 6.0]),
            ContinuousInput(key="x5", bounds=[5.0, 6.0]),
        ]
    )

    output_features = Outputs(
        features=[
            ContinuousOutput(
                key=f"y{i}", objective=MinimizeObjective(w=1.0, bounds=[0.0, 1.0])
            )
            for i in range(n_obj)
        ]
    )

    domain = Domain(inputs=input_features, outputs=output_features)

    return domain


In [ ]:
from bofire.strategies.predictives.enting import domain_to_problem_config
import entmoot.benchmarks as ent
from entmoot.problem_config import ProblemConfig

domain = build_multi_obj_categorical_problem()
problem_config, _ = domain_to_problem_config(domain)
problem_config_ent = ProblemConfig()
ent.build_multi_obj_categorical_problem(problem_config_ent)

print(problem_config, "\n", problem_config_ent)